In [1]:
import numpy as np
import rasterio.fill
import sklearn.neighbors

from parsers.utils import pcloud
from parsers.utils import raster

STEP 1: Read the point cloud into memory.

In [3]:
pc = pcloud.PointCloud("C:/Documents/RoofSense/feature/parsers/test.laz")

STEP 2: Initialize the raster grid and compute the center coordinates of its cells.

In [4]:
# TODO: Promote this static constant to an environment variable.
CELL_SIZE = 0.25

# TODO: Write a method to compute the bounds of the point cloud.
r = raster.Raster(CELL_SIZE, pc.bbox())

In [6]:
# TODO: Check how the tree behaves when duplicate points are inserted to it.
index = sklearn.neighbors.KDTree(np.vstack((pc.las.x, pc.las.y)).transpose())

STEP 3: Build a k-d tree using the 2D projection of the point cloud and find the neighbors of each cell.

In [7]:
# TODO: Optimize this tree creation and indexing processes.
neighbors, distances = index.query_radius(r.xy(), r=CELL_SIZE, return_distance=True)

STEP 4: Interpolate the DSM and reflectance fields.

In [8]:
# FIXME: Return the neighbor indices and corresponding weights instead of performing the
#        interpolation directly.
# TODO: Optimize this loop.
scalars = "z", "Reflectance"
rasters = {scalar: raster.Raster(CELL_SIZE, pc.bbox()) for scalar in scalars}

nodata = []
for cell_id, cell_nb in enumerate(neighbors):
    if len(cell_nb) == 0:
        nodata.append(cell_id)
        continue
    attribs = {scalar: [] for scalar in scalars}
    for scalar in scalars:
        attribs[scalar].append(pc.las.points[cell_nb][scalar])
    row, col = np.divmod(cell_id, r.width)
    if np.any(distances[cell_id] == 0):
        # The cell center belongs to the point cloud.
        nb_id = cell_nb[np.argsort(distances[cell_id])[0]]
        # NOTE: This indexing notation results in the point being returned in a familiar
        #       format.
        for scalar in scalars:
            rasters[scalar][row, col] = pc.las.points[[nb_id]][scalar].scaled_array()
    else:
        weights = distances[cell_id] ** -2
        for scalar in scalars:
            rasters[scalar][row, col] = np.sum(attribs[scalar] * weights) / np.sum(weights)

In [14]:
mask = np.ones((r.height, r.width))
mask[np.divmod(nodata, r.width)] = 0
mask[mask==0]

array([0., 0., 0., ..., 0., 0., 0.])

In [20]:
mask = np.ones((r.height, r.width))
mask[np.divmod(nodata, r.width)] = 0

for scalar, raster_ in rasters.items():
    raster_._data = rasterio.fill.fillnodata(raster_._data, mask)
    rasters[scalar].save(f"{scalar}.tif")

STEP 5: Compute the slope from the DSM.

In [21]:
rasters["z"].slope().save("Slope.tif")